In [1]:
import pandas
from IPython.display import clear_output
from datetime import datetime
import json
from collections import defaultdict

In [2]:
from classes import Question
import classes
import look_up_utils
import createq_utils
import pandas
from display_utils import display_question

## Set parameter values

In [3]:
df = pandas.read_pickle('../EventRegistries/GunViolenceArchive/frames/all')
confusion_tuple = ('location', 'time')
min_num_answer_incidents = 2

## Load frames

## Load look_up

In [4]:
look_up, parameters2incident_uris = look_up_utils.create_look_up(df, 
                                                                 discard_ambiguous_names=True,
                                                                 allowed_incident_years={2017},
                                                                 check_name_in_article=True)

## Load questions

In [5]:
candidates=createq_utils.lookup_and_merge(look_up, 
                                          parameters2incident_uris,
                                          confusion_tuple,
                                          min_num_answer_incidents,
                                          df,
                                          debug=False,
                                          inspect_one=False,
                                          set_attr_values=True) 
print({c.granularity for c in candidates})

{('address', 'month'), ('city', 'day'), ('state', 'month'), ('city', 'month'), ('address', 'year'), ('state', 'year'), ('state', 'day'), ('city', 'year')}


## Display a question

In [6]:
print('the available attributes are:\n')
for candidate in candidates:
    for prop in dir(candidate):
        if all([not prop.startswith('_'),
                prop not in {'debug', 'confusion_df', 'answer_df'}]):
            print(prop)
    break

the available attributes are:

a_avg_date_spread
a_avg_num_sources
a_sources
answer
answer_incident_uris
c2s_ratio
c_avg_date_spread
c_avg_num_sources
c_sources
confusion_factors
confusion_incident_uris
gold_loc_meaning
granularity
gvdb_annotations
loc_oa
location_confusion
meanings
num_a_sources
num_both_sf_overlap
num_c_sources
num_gvdb_part_annotations
oa
oa_info
participant_confusion
q_id
question
set_all_attributes
sf
time_confusion
time_oa
to_conll
to_conll_one_file_per_question
to_include_in_task


In [7]:
wanted_attrs = [
    'q_id',
    'question',
    'granularity',
    'answer_incident_uris',
    'num_both_sf_overlap',
    #'confusion_incident_uris',
    'meanings',
    'answer',
    'c2s_ratio',
    'location_confusion',
    'time_confusion',
    'participant_confusion',
    'a_avg_num_sources',
    'a_avg_date_spread',
    'c_avg_num_sources',
    'c_avg_date_spread',
    'num_gvdb_part_annotations',
    'loc_oa',
    'time_oa'
]

#for candidate in candidates:
 #   clear_output()
 #   display_question(candidate, wanted_attrs)
 #   break
    #input('next?')

In [8]:
%%bash
rm -rf trial
mkdir trial
mkdir trial/system_input
mkdir trial/system_output
ls trial/
cp ../Installing/setup.sh trial
cp obtain_task_data.py trial

system_input
system_output


In [9]:
output_folder = 'trial'
subtask = 'S2'
event_types = ['killing', 'injuring']

questions = dict()
answers = dict()

maximum = 100
counter = 0
for candidate in candidates:
    
    if 'address' in candidate.granularity:
        continue 
        
        
    counter += 1
    if counter == maximum:
        break
        
        
    dfs = [('gold', candidate.answer_df), 
           #('confusion', candidate.confusion_df)
          ]
    output_path = '%s/system_input/%s.conll' % (output_folder, candidate.q_id)
    candidate.to_conll_one_file_per_question(dfs, output_path)
    one_question = candidate.question(subtask, event_types)
    
    if len(candidate.all_doc_ids) != candidate.answer:
        print('mismatch answer and incidents/documents')

    if all([one_question is not None,
            len(candidate.all_doc_ids) >= 2]):
        questions[candidate.q_id] = one_question
        answers[candidate.q_id] = {'numerical_answer': len(candidate.all_doc_ids),
                                   'answer_docs' : candidate.all_doc_ids}

question_out_path = '{output_folder}/questions.json'.format_map(locals())
with open(question_out_path, 'w') as outfile:
    outfile.write(json.dumps(questions, indent=4, sort_keys=True))
    
answers_out_path = '{output_folder}/answers.json'.format_map(locals())
with open(answers_out_path, 'w') as outfile:
    outfile.write(json.dumps(answers, indent=4, sort_keys=True))

mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
mismatch answer and incidents/documents
multiple or no dbpedia options for 411: set()
mismatch answer and incidents/documents
multiple or no dbpedia options for 428: set()
mismatch answer and incidents/documents
multiple or no dbpedia options for 510: set()
mismatch answer and incidents/documents
multiple or no dbpedia options for 81: {'http://dbpedia.org/resource/Indiana', 'http://dbpedia.org/res